In [1]:
prompt = """Look at this food image and provide a recipe in XML format.
    Use the following structure:
    <recipe>
      <title>The dish name</title>
      <ingredients>
        <ingredient>1 cup whole milk</ingredient>
        <ingredient>2 tbsp sugar</ingredient>
        <!-- Add all ingredients in this format -->
      </ingredients>
      <instructions>
        <step>1. Preheat the oven to 350°F.</step>
        <step>2. Mix all ingredients in a bowl.</step>
        <!-- Add all steps in this format -->
      </instructions>
    </recipe>
    
    Important: 
    1. Make sure to format your response as valid XML following the exact structure above.
    2. For each ingredient, include quantity, unit, and name in the <ingredient> tag.
    3. For each instruction, include the step number and description in the <step> tag.
    4. Do not use commas to separate ingredients - use individual <ingredient> tags.
    5. Do not use newlines or commas inside the instruction text.
    """

In [2]:
%pip install pandas datasets

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install fuzzywuzzy python-Levenshtein

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from datasets import Dataset
import os


# File paths
csv_file_path = "inverse_cooking_dataset/Food Ingredients and Recipe Dataset with Image Name Mapping.csv"
image_dir_path = "inverse_cooking_dataset/Food Images/Food Images"

# Load the CSV file directly
print(f"Loading CSV from: {csv_file_path}")
df = pd.read_csv(csv_file_path)

# Add the full image paths
if 'Image_Name' in df.columns:
    print(f"Adding image paths from: {image_dir_path}")
    df['full_image_path'] = df['Image_Name'].apply(lambda x: os.path.join(image_dir_path, f"{x}.jpg"))
    
    # Verify a few paths (optional)
    print(f"First few image paths: {df['full_image_path'].head(2).tolist()}")

# Convert to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df)
print(f"Dataset created with {len(hf_dataset)} examples")
print(f"Sample data: {hf_dataset[0]}")

/Users/juyounglee/Desktop/School/2025Spring/csci467/inverse_cooking/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading CSV from: inverse_cooking_dataset/Food Ingredients and Recipe Dataset with Image Name Mapping.csv
Adding image paths from: inverse_cooking_dataset/Food Images/Food Images
First few image paths: ['inverse_cooking_dataset/Food Images/Food Images/miso-butter-roast-chicken-acorn-squash-panzanella.jpg', 'inverse_cooking_dataset/Food Images/Food Images/crispy-salt-and-pepper-potatoes-dan-kluger.jpg']
Dataset created with 13501 examples
Sample data: {'Unnamed: 0': 0, 'Title': 'Miso-Butter Roast Chicken With Acorn Squash Panzanella', 'Ingredients': '[\'1 (3½–4-lb.) whole chicken\', \'2¾ tsp. kosher salt, divided, plus more\', \'2 small acorn squash (about 3 lb. total)\', \'2 Tbsp. finely chopped sage\', \'1 Tbsp. finely chopped rosemary\', \'6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature\', \'¼ tsp. ground allspice\', \'Pinch of crushed red pepper flakes\', \'Freshly ground black pepper\', \'⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)\', 

In [5]:
from utils import preprocess_dataset

preprocessed_dataset = preprocess_dataset(hf_dataset)

Preprocessing dataset with 13501 examples...


Map:  28%|██▊       | 3778/13501 [00:01<00:02, 3339.77 examples/s]

Map:  35%|███▍      | 4686/13501 [00:01<00:02, 3516.27 examples/s]

Map:  44%|████▍     | 5914/13501 [00:02<00:03, 2165.99 examples/s]

Map:  53%|█████▎    | 7193/13501 [00:02<00:02, 2496.33 examples/s]

...

Map:  71%|███████▏  | 9628/13501 [00:03<00:01, 3535.94 examples/s]

Map:  77%|███████▋  | 10384/13501 [00:03<00:00, 3540.45 examples/s]

Map:  89%|████████▉ | 12000/13501 [00:04<00:00, 3399.55 examples/s]

Filter: 100%|██████████| 13501/13501 [00:00<00:00, 32493.22 examples/s]

Preprocessing complete. 13471 examples with valid images (filtered out 30 examples)


In [6]:
preprocessed_dataset.features


{'Unnamed: 0': Value(dtype='int64', id=None),
 'Title': Value(dtype='string', id=None),
 'Ingredients': Value(dtype='string', id=None),
 'Instructions': Value(dtype='string', id=None),
 'Image_Name': Value(dtype='string', id=None),
 'Cleaned_Ingredients': Value(dtype='string', id=None),
 'full_image_path': Value(dtype='string', id=None),
 'parsed_ingredients': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'parsed_cleaned_ingredients': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'instruction_steps': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'base64_image': Value(dtype='string', id=None)}

In [7]:
preprocessed_dataset.to_pandas().head()

,Unnamed: 0,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,full_image_path,parsed_ingredients,parsed_cleaned_ingredients,instruction_steps,base64_image
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...",inverse_cooking_dataset/Food Images/Food Image...,"[1 (3½–4-lb.) whole chicken, 2¾ tsp. kosher sa...","[1 (3½–4-lb.) whole chicken, 2¾ tsp. kosher sa...","[Pat chicken dry with paper towels, season all...",/9j/4AAQSkZJRgABAQAAAQABAAD/4gIcSUNDX1BST0ZJTE...
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (...",inverse_cooking_dataset/Food Images/Food Image...,"[2 large egg whites, 1 pound new potatoes (abo...","[2 large egg whites, 1 pound new potatoes (abo...",[Preheat oven to 400°F and line a rimmed bakin...,/9j/4AAQSkZJRgABAQAAAQABAAD/4gIcSUNDX1BST0ZJTE...
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ...",inverse_cooking_dataset/Food Images/Food Image...,"[1 cup evaporated milk, 1 cup whole milk, 1 ts...","[1 cup evaporated milk, 1 cup whole milk, 1 ts...",[Place a rack in middle of oven; preheat to 40...,/9j/4AAQSkZJRgABAQAAAQABAAD/4gIcSUNDX1BST0ZJTE...
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in...",inverse_cooking_dataset/Food Images/Food Image...,"[1 (¾- to 1-pound) round Italian loaf, cut int...","[1 (¾- to 1-pound) round Italian loaf, cut int...",[Preheat oven to 350°F with rack in middle. Ge...,/9j/4AAQSkZJRgABAQAAAQABAAD/4gIcSUNDX1BST0ZJTE...
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",inverse_cooking_dataset/Food Images/Food Image...,"[1 teaspoon dark brown sugar, 1 teaspoon hot w...","[1 teaspoon dark brown sugar, 1 teaspoon hot w...",[Stir together brown sugar and hot water in a ...,/9j/4AAQSkZJRgABAQAAAQABAAD/4gIcSUNDX1BST0ZJTE...


In [8]:
test_dataset = preprocessed_dataset.select(range(3))

In [9]:
# generate recipe for each example
from utils import generate_response, parse_recipe_xml
responses = []
for example in test_dataset:
    input_image = example['base64_image']
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{input_image}",
                    },
                },
            ],
        }
    ]
    response = generate_response(messages)
    responses.append(response)


In [10]:
y_pred = []
for response in responses:
    parsed_recipe = parse_recipe_xml(response)
    y_pred.append(parsed_recipe)

In [11]:
y_pred

[{'title': 'Roasted Chicken with Fall Vegetables',
  'ingredients': ['1 whole chicken (3-4 lbs)',
   '2 tbsp olive oil',
   '1 tsp salt',
   '1 tsp pepper',
   '1 tsp dried thyme',
   '1 tsp dried rosemary',
   '1 large onion, peeled and chopped',
   '2 carrots, peeled and chopped',
   '2 celery stalks, chopped',
   '2 large sweet potatoes, peeled and chopped',
   '2 large parsnips, peeled and chopped',
   '1 large apple, cored and chopped',
   '2 tbsp butter'],
  'steps': ['1. Preheat the oven to 425°F.',
   '2. Rinse the chicken and pat it dry with paper towels.',
   '3. In a small bowl, mix together the olive oil, salt, pepper, thyme, and rosemary.',
   '4. Rub the mixture all over the chicken, making sure to get some under the skin as well.',
   '5. Place the chicken in a roasting pan and put it in the oven.',
   '6. Roast the chicken for 45-50 minutes, or until it reaches an internal temperature of 165°F.',
   '7. While the chicken is roasting, toss the chopped vegetables with the

In [12]:
for i in range(len(y_pred)):
    print(f"Example {i+1}:")
    print(f"Title: {y_pred[i]['title']}")
    print("Ingredients:")
    for ingredient in y_pred[i]['ingredients']:
        print(f"- {ingredient}")
    print("Instructions:")
    for step in y_pred[i]['steps']:
        print(f"{step}")

Example 1:
Title: Roasted Chicken with Fall Vegetables
Ingredients:
- 1 whole chicken (3-4 lbs)
- 2 tbsp olive oil
- 1 tsp salt
- 1 tsp pepper
- 1 tsp dried thyme
- 1 tsp dried rosemary
- 1 large onion, peeled and chopped
- 2 carrots, peeled and chopped
- 2 celery stalks, chopped
- 2 large sweet potatoes, peeled and chopped
- 2 large parsnips, peeled and chopped
- 1 large apple, cored and chopped
- 2 tbsp butter
Instructions:
1. Preheat the oven to 425°F.
2. Rinse the chicken and pat it dry with paper towels.
3. In a small bowl, mix together the olive oil, salt, pepper, thyme, and rosemary.
4. Rub the mixture all over the chicken, making sure to get some under the skin as well.
5. Place the chicken in a roasting pan and put it in the oven.
6. Roast the chicken for 45-50 minutes, or until it reaches an internal temperature of 165°F.
7. While the chicken is roasting, toss the chopped vegetables with the butter and spread them out in a single layer on a baking sheet.
8. Roast the vegetabl

In [13]:
%pip install nltk rouge-score sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [16]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/juyounglee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/juyounglee/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [17]:
from evals import compute_evals
result = []

for  in test_dataset:
    
pred_recipe = y_pred[0]
golden_recipe = test_dataset[0]

result = compute_evals(pred_recipe, golden_recipe)
result

{'cosine_similarity': {'steps': np.float32(0.64165723),
  'ingredients': np.float32(0.6825431)},
 'bleu_score': {'steps': 8.041251819480524e-11,
  'ingredients': 0.06097986499362348},
 'rouge_scores': {'steps': {'rouge1': 0.05184968834432905,
   'rougeL': 0.0346136771604631},
  'ingredients': {'rouge1': 0.46401376401376393,
   'rougeL': 0.4383727383727384}}}